# Sales Report📊

## Install & import the dependencies

In [26]:
from datetime import date
from pathlib import Path
import sqlite3

import pandas as pd  # pip install pandas
import plotly.express as px  # pip install plotly-express kaleido
from fpdf import FPDF  # pip install fpdf

## Define paths and chart style

In [27]:
# Define the plotly template. Some other examples:
# "plotly", "ggplot2", "seaborn", "simple_white", "plotly_dark", "plotly_white", ..
plotly_template = "presentation"

In [28]:
# Define the paths
current_dir = Path(__file__).parent if "__file__" in locals() else Path.cwd()
database_path = current_dir / "sales.db"
output_dir = current_dir / "output"

# Create the output directory and its parent directory if they do not exist
output_dir.mkdir(parents=True, exist_ok=True)

## Total Sales by Month

In [29]:
# Create a connection to the database
conn = sqlite3.connect(database_path)

In [30]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('sales',), ('products',), ('customers',)]


In [31]:
pd.read_sql_query(query, conn)

,customer_name,total_sales
0,David Brown,32075
1,Bob Smith,31725
2,Grace Lee,29550
3,Frank Wilson,29315
4,Jane Doe,27880
5,Alice Jones,27740
6,Henry Chen,27655
7,Isabel Garcia,25375
8,John Doe,25335
9,Emily Davis,24820


In [32]:
# Execute the query and load results into a Pandas DataFrame
query = '''
SELECT sale_date, SUM(total_price) as total_sales
FROM sales
GROUP BY sale_date
ORDER BY sale_date ASC
'''
df = pd.read_sql_query(query, conn)

In [33]:
# Print the DataFrame
df.head()

,sale_date,total_sales
0,2022-01-01,1265
1,2022-01-03,595
2,2022-01-04,305
3,2022-01-06,525
4,2022-01-07,1175


In [34]:
# Check the data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332 entries, 0 to 331
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sale_date    332 non-null    object
 1   total_sales  332 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.3+ KB


In [35]:
# Convert sale_date to datetime
df['sale_date'] = pd.to_datetime(df['sale_date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332 entries, 0 to 331
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   sale_date    332 non-null    datetime64[ns]
 1   total_sales  332 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 5.3 KB


In [36]:
# Set the sale_date column as the index
df = df.set_index('sale_date')
df.head(3)

,total_sales
sale_date,
2022-01-01,1265
2022-01-03,595
2022-01-04,305


In [37]:
# Resample the data to a monthly frequency and compute the sum
df_monthly = df.resample('M').sum()
df_monthly

,total_sales
sale_date,
2022-01-31,24795
2022-02-28,20605
2022-03-31,27565
2022-04-30,22080
2022-05-31,21915
2022-06-30,23670
2022-07-31,27540
2022-08-31,22525
2022-09-30,20850


In [38]:
# Map the month number to short month name
df_monthly['month_name'] = df_monthly.index.strftime('%b')
df_monthly

,total_sales,month_name
sale_date,,
2022-01-31,24795,Jan
2022-02-28,20605,Feb
2022-03-31,27565,Mar
2022-04-30,22080,Apr
2022-05-31,21915,May
2022-06-30,23670,Jun
2022-07-31,27540,Jul
2022-08-31,22525,Aug
2022-09-30,20850,Sep


In [39]:
# Create the Plotly figure with text parameter
fig = px.bar(df_monthly,
             x='month_name',
             y='total_sales',
             template=plotly_template,
             text='total_sales')

# Set the layout
fig.update_layout(
    title='Total Sales by Month',
    xaxis_title='Month',
    yaxis_title='Total Sales ($)',
    yaxis_tickprefix='$',
)

# Show the plot
fig.show()

# Save the chart as a PNG image
fig.write_image(output_dir / 'monthly_sales.png',
                width=1200,
                height=400,
                scale=4)

## Total Sales by Product

In [40]:
# Execute the query and load results into a Pandas DataFrame
query = '''
SELECT p.product_name, SUM(s.total_price) as total_sales
FROM sales s
JOIN products p ON s.product_id = p.product_id
GROUP BY p.product_name
'''
df = pd.read_sql_query(query, conn)

In [41]:
df

,product_name,total_sales
0,Product A,56100
1,Product B,26000
2,Product C,81450
3,Product D,42440
4,Product E,75480


In [42]:
# Create the Plotly figure with text parameter
fig = px.bar(df,
             x='product_name',
             y='total_sales',
             template=plotly_template,
             text='total_sales')

# Set the layout
fig.update_layout(
    title='Total Sales by Product',
    xaxis_title='Product',
    yaxis_title='Total Sales ($)',
    yaxis_tickprefix='$',
)

# Show the plot
fig.show()

# Save the chart as a PNG image
fig.write_image(output_dir / 'product_sales.png',
                width=1200,
                height=400,
                scale=4)

## Top Customer by Sales

In [43]:
# Execute the query and load results into a Pandas DataFrame
query = '''
SELECT c.first_name || ' ' || c.last_name as customer_name, SUM(s.total_price) as total_sales
FROM sales s
JOIN customers c ON s.customer_id = c.customer_id
GROUP BY customer_name
ORDER BY total_sales DESC
LIMIT 10
'''
df = pd.read_sql_query(query, conn)

In [44]:
df

,customer_name,total_sales
0,David Brown,32075
1,Bob Smith,31725
2,Grace Lee,29550
3,Frank Wilson,29315
4,Jane Doe,27880
5,Alice Jones,27740
6,Henry Chen,27655
7,Isabel Garcia,25375
8,John Doe,25335
9,Emily Davis,24820


In [45]:
# Create the Plotly figure
fig = px.bar(df,
             x='customer_name',
             y='total_sales',
             template=plotly_template,
             text='total_sales')

# Set the layout
fig.update_layout(
    title='Top Customers by Sales',
    xaxis_title='Customer',
    yaxis_title='Total Sales ($)',
    yaxis_tickprefix='$',
)

# Show the plot
fig.show()

# Save the chart as a PNG image
fig.write_image(output_dir / 'customer_sales.png',
                width=1200,
                height=400,
                scale=4)

## Create PDF Report

In [46]:
# Define the font color as RGB values (dark gray)
font_color = (64, 64, 64)

# Find all PNG files in the output folder
chart_filenames = [str(chart_path) for chart_path in output_dir.glob("*.png")]

# Create a PDF document and set the page size
pdf = FPDF()
pdf.add_page()
pdf.set_font('Arial', 'B', 24)

# Add the overall page title
title = f"Sales Report as of {date.today().strftime('%m/%d/%Y')}"
pdf.set_text_color(*font_color)
pdf.cell(0, 20, title, align='C', ln=1)

# Add each chart to the PDF document
for chart_filename in chart_filenames:
    pdf.ln(10)  # Add padding at the top of the next chart
    pdf.image(chart_filename, x=None, y=None, w=pdf.w - 20, h=0)

# Save the PDF document to a file on disk
pdf.output(output_dir / "sales_report.pdf", "F")

''

## Bonus - SQL vs. Pandas

### Customer Segment using SQL

In [47]:
# Execute the query and load results into a Pandas DataFrame
query = '''
SELECT 
  customers.customer_id, 
  customers.first_name || ' ' || customers.last_name as customer_name, 
  SUM(sales.total_price) as total_sales,
  CASE 
    WHEN SUM(sales.total_price) > 30000 THEN 'High Value'
    WHEN SUM(sales.total_price) > 26000 THEN 'Medium Value'
    ELSE 'Low Value'
  END as customer_segment
FROM sales
INNER JOIN customers ON sales.customer_id = customers.customer_id
GROUP BY customers.customer_id
ORDER BY total_sales DESC
'''

df = pd.read_sql_query(query, conn)
df

,customer_id,customer_name,total_sales,customer_segment
0,5,David Brown,32075,High Value
1,3,Bob Smith,31725,High Value
2,8,Grace Lee,29550,Medium Value
3,7,Frank Wilson,29315,Medium Value
4,2,Jane Doe,27880,Medium Value
5,4,Alice Jones,27740,Medium Value
6,9,Henry Chen,27655,Medium Value
7,10,Isabel Garcia,25375,Low Value
8,1,John Doe,25335,Low Value
9,6,Emily Davis,24820,Low Value


### Customer Segment using SQL + Pandas

In [48]:
# Execute the query and load results into a Pandas DataFrame
query = '''
SELECT 
  customers.customer_id, 
  customers.first_name || ' ' || customers.last_name as customer_name, 
  SUM(sales.total_price) as total_sales
FROM sales
INNER JOIN customers ON sales.customer_id = customers.customer_id
GROUP BY customers.customer_id
'''

df = pd.read_sql_query(query, conn)
df

,customer_id,customer_name,total_sales
0,1,John Doe,25335
1,2,Jane Doe,27880
2,3,Bob Smith,31725
3,4,Alice Jones,27740
4,5,David Brown,32075
5,6,Emily Davis,24820
6,7,Frank Wilson,29315
7,8,Grace Lee,29550
8,9,Henry Chen,27655
9,10,Isabel Garcia,25375


In [49]:
# Group the data by customer segment
bins = [0, 26000, 30000, float('inf')]
labels = ['Low Value', 'Medium Value', 'High Value']
df['customer_segment'] = pd.cut(df['total_sales'], bins=bins, labels=labels)

# Order the data by total sales
df = df.sort_values(by='total_sales', ascending=False)
df

,customer_id,customer_name,total_sales,customer_segment
4,5,David Brown,32075,High Value
2,3,Bob Smith,31725,High Value
7,8,Grace Lee,29550,Medium Value
6,7,Frank Wilson,29315,Medium Value
1,2,Jane Doe,27880,Medium Value
3,4,Alice Jones,27740,Medium Value
8,9,Henry Chen,27655,Medium Value
9,10,Isabel Garcia,25375,Low Value
0,1,John Doe,25335,Low Value
5,6,Emily Davis,24820,Low Value


In [50]:
# Close the connection
conn.close()